# GAN IMPLEMENTATION FROM COMMUNITY INQ28 MODELS
- Code Author: Jack MacCormick
- Instagram: argelpaints: https://www.instagram.com/argelpaints/
- Done in partnership with 28MAG: https://28-mag.com/

**GAN Training and Development Code:**

This project is the use of GAN style Machine Learning algorithms to generate new images, based off the works of the Warhammer INQ28 community. These images are the dreamings of an abhorent AI system, who's task is to understand "what is INQ28" it knows not the world, it's surroundings, only those offered to it, and the endless void

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
# import necessary packages
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pylab
import h5py

from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import random
from numpy.random import randint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

from tqdm import tqdm 
from IPython import display 

In [69]:
"""
define the standalone discriminator model
key value is the learning rate lr

"""

def define_discriminator(in_shape=(200,200,1)):
	# weight initialization
	init = 'uniform'
	# define model
	model = Sequential()
	# downsample to 100x100
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 50x50
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0001, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [70]:
def define_generator(latent_dim):
	# weight initialization
	init = 'uniform'
	# define model
	model = Sequential()
	# foundation for 50x50 image
	n_nodes = 256 * 50 * 50
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((50, 50, 256)))
	# upsample to 100x100
	model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 200x200
	model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model

In [113]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.7) # lr=00015 seems to be the ideal value for training
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [72]:
# default values of standardised images
preferred_width = 200
preferred_height = 200

In [73]:
pathSource = '/content/drive/MyDrive/BnWDrive'
dir_list = os.listdir(pathSource)

In [74]:


images = []
# for image in pathSource, convert them to BnW, save that to pathDest, then add standardised version to images[]
for image in dir_list:
    img = Image.open(pathSource+"/"+image)
    imgArray = np.array(img) # int array of pixels
    imgArray = imgArray.astype(float)/255 # standardise values to be 0:255
    images.append(imgArray)


In [75]:
# setting a param for proportion of data to keep for testing
testingSize = int(len(dir_list)/5)

X_testing = np.array(images[0:testingSize])
X_training = np.array(images[testingSize:len(images)])

"""
# this is all diagnostic checks to ensuredata is passed correctly to training and testing splits

# code:
print(np.shape(X_testing)) 
print(X_testing[1]) # checking it's correctly put something into the testing variable
print(np.shape(X_training))
# end code
"""

"\n# this is all diagnostic checks to ensuredata is passed correctly to training and testing splits\n\n# code:\nprint(np.shape(X_testing)) \nprint(X_testing[1]) # checking it's correctly put something into the testing variable\nprint(np.shape(X_training))\n# end code\n"

In [76]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y

In [77]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.uniform(low=0.0, high=1.0, size=(latent_dim, n_samples))
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [78]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = abs(generate_latent_points(latent_dim, n_samples))
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n_samples, 1))
	return X, y

In [114]:
"""
trains the generator and discriminator parts of the GAN
saves the net model to a .h5 file
"""

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs, n_batch):
	# calculate the number of batches per epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the total iterations based on batch and epoch
	n_steps = bat_per_epo * n_epochs
	# calculate the number of samples in half a batch
	half_batch = int(n_batch / 2)
	# prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
		# generate 'fake' examples
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d/%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
			(i+1, n_steps, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
		# record history
		d1_hist.append(d_loss1)
		d2_hist.append(d_loss2)
		g_hist.append(g_loss)
		a1_hist.append(d_acc1)
		a2_hist.append(d_acc2)
		if (i%10==0): # save frequency, this should really be a smart value
			g_model.save('WARgenerator.h5')		
	print("finished training")
	# this is the generator that can be called externally
	g_model.save('WARgenerator.h5') 
	"""
	Save is also embed this within the loop every so many itterations
	so something is saved/retained if user needs to abort while long training processes are occuring or if something fails, if the kernel or local machine crashes or if local machine looses power
	"""

In [80]:
"""
set up the data into something the GAN can use in training
"""

X_train=np.reshape(X_training,(X_training.shape[0], X_training.shape[1],X_training.shape[2],1))

In [124]:
"""
set up for training and training call
this can take some time depending on number of epochs
recommend running on a GPU
"""

# training params
latent_dim = 50 # size of the latent space
# training above 50 generations, results stabilise, and no change is noticable
n_epochs = 40 # number of "generations"
n_batch = 15 # batch (of images) size
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = X_train

# train model
train(generator, discriminator, gan_model, dataset, latent_dim, n_epochs, n_batch)

>1/400, d1=0.622, d2=0.740 g=0.642, a1=100, a2=0
>2/400, d1=0.302, d2=1.122 g=0.425, a1=100, a2=0
>3/400, d1=0.199, d2=1.519 g=0.351, a1=100, a2=0
>4/400, d1=0.272, d2=1.417 g=0.441, a1=100, a2=0
>5/400, d1=0.450, d2=1.089 g=0.592, a1=100, a2=0
>6/400, d1=0.621, d2=0.878 g=0.699, a1=100, a2=0
>7/400, d1=0.733, d2=0.821 g=0.702, a1=14, a2=0
>8/400, d1=0.753, d2=0.888 g=0.632, a1=0, a2=0
>9/400, d1=0.765, d2=1.000 g=0.521, a1=14, a2=0
>10/400, d1=0.702, d2=1.137 g=0.424, a1=42, a2=0
>11/400, d1=0.653, d2=1.297 g=0.333, a1=85, a2=0
>12/400, d1=0.565, d2=1.416 g=0.296, a1=100, a2=0
>13/400, d1=0.455, d2=1.428 g=0.282, a1=100, a2=0
>14/400, d1=0.343, d2=1.532 g=0.261, a1=100, a2=0
>15/400, d1=0.306, d2=1.687 g=0.230, a1=100, a2=0
>16/400, d1=0.283, d2=1.704 g=0.232, a1=100, a2=0
>17/400, d1=0.321, d2=1.685 g=0.257, a1=100, a2=0
>18/400, d1=0.358, d2=1.536 g=0.314, a1=100, a2=0
>19/400, d1=0.444, d2=1.422 g=0.366, a1=100, a2=0
>20/400, d1=0.500, d2=1.378 g=0.368, a1=100, a2=0
>21/400, d1=0.5

In [125]:
"""
getting the model to draw something,
this can be done seperately, if put in another file
"""

model = load_model('WARgenerator.h5', compile = False)
# generate images
latent_points = generate_latent_points(latent_dim, 1)
 # has quite the think about it
predictionInstance = model.predict(latent_points)
print("done thinking")

done thinking


In [131]:
"""
Training of existing model file:
May not always want to run this cell
"""


n_epochs = 50
n_batch = 15
train(model, discriminator, gan_model, dataset, latent_dim, n_epochs, n_batch)

()

>1/500, d1=0.724, d2=0.719 g=0.705, a1=0, a2=0
>2/500, d1=0.718, d2=0.689 g=0.702, a1=0, a2=100
>3/500, d1=0.710, d2=0.662 g=0.700, a1=14, a2=100
>4/500, d1=0.716, d2=0.644 g=0.680, a1=14, a2=100
>5/500, d1=0.711, d2=0.629 g=0.658, a1=0, a2=100
>6/500, d1=0.692, d2=0.614 g=0.632, a1=42, a2=100
>7/500, d1=0.691, d2=0.600 g=0.604, a1=42, a2=100
>8/500, d1=0.663, d2=0.582 g=0.571, a1=100, a2=100
>9/500, d1=0.674, d2=0.572 g=0.525, a1=85, a2=100
>10/500, d1=0.633, d2=0.563 g=0.480, a1=100, a2=100
>11/500, d1=0.626, d2=0.549 g=0.430, a1=100, a2=100
>12/500, d1=0.634, d2=0.534 g=0.386, a1=100, a2=100
>13/500, d1=0.592, d2=0.521 g=0.333, a1=100, a2=100
>14/500, d1=0.558, d2=0.513 g=0.284, a1=100, a2=100
>15/500, d1=0.550, d2=0.503 g=0.236, a1=100, a2=100
>16/500, d1=0.494, d2=0.484 g=0.199, a1=100, a2=100
>17/500, d1=0.532, d2=0.469 g=0.157, a1=100, a2=100
>18/500, d1=0.407, d2=0.458 g=0.124, a1=100, a2=100
>19/500, d1=0.366, d2=0.432 g=0.100, a1=100, a2=100
>20/500, d1=0.387, d2=0.402 g=0.07

()

In [132]:
"""
getting the prediction directly from generator
functionally the same as prediction instance
"""

xout = generate_fake_samples(generator, latent_dim, n_samples = 1)[0]
# predictionInstance = model.predict(latent_points)
# convert floats from ranges 0:1 to 0:255, so they can be blackness values in an image
xout = predictionInstance*255 
# print(xout) # this is diagnostic
# np.shape(xout) # this is diagnostic
flatXOut = xout.flatten()  
# print(len(flatXOut)) # should be 40,000 # this is diagnostic
arr_2d = np.reshape(flatXOut, (200, 200))
# convert float array to ints
array_int = abs(np.array(arr_2d, dtype='int')) 
print(np.shape(array_int)) # this is diagnostic
array_int # this is diagnostic



(200, 200)


array([[232, 240, 246, ..., 239, 252, 250],
       [227, 244, 246, ..., 245, 249, 251],
       [228, 237, 199, ..., 249, 245, 247],
       ...,
       [230, 240, 239, ..., 246, 249, 252],
       [244, 234, 232, ..., 252, 248, 249],
       [249, 246, 216, ..., 248, 248, 151]])

In [128]:
'''
"""
getting the prediction into something useful
"""

# convert floats from ranges 0:1 to 0:255, so they can be blackness values in an image
xout = predictionInstance*255 
# print(xout) # this is diagnostic
# np.shape(xout) # this is diagnostic
flatXOut = xout.flatten()  
# print(len(flatXOut)) # should be 40,000 # this is diagnostic
arr_2d = np.reshape(flatXOut, (200, 200))
# convert float array to ints
array_int = abs(np.array(arr_2d, dtype='int')) 
# print(np.shape(array_int)) # this is diagnostic
# array_int # this is diagnostic

'''
()

()

In [133]:
"""
draw the image of the output array
"""

img = Image.fromarray(array_int.astype(np.uint8))

"""
diagnostic check of the output image

# code:
print("Format: {0}\nSize: {1}\nMode: {2}".format(img.format, 
    img.size, img.mode)) 
# end code
"""


'\ndiagnostic check of the output image\n\n# code:\nprint("Format: {0}\nSize: {1}\nMode: {2}".format(img.format, \n    img.size, img.mode)) \n# end code\n'

In [134]:
"""
convert the image file to a recognisable and portable format
write out the image file to somewhere in the local machine
"""


pathOut = '/content/drive/MyDrive/BnWOut/lr=0.0002 - 500'
imgRGB = img.convert(mode="RGB")
imgRGB.save(pathOut+"/dreamParamTest10.jpg")